# Create Train and Test Folds

In [ ]:
#libraries
import pandas as pd
import numpy as np
import os
import sys
import shutil
import glob
import joblib
import warnings
from sklearn.model_selection import GroupKFold


In [ ]:
tables_path ='/media/hcs-sci-psy-narun/ABCC/fmriresults01/derivatives/ML_Tables/'

In [ ]:
# Identify subjects with bad eye sight to remove them. 
svs = pd.DataFrame(pd.read_csv('/media/hcs-sci-psy-narun/abcd-data-release-5.0/core/neurocognition/nc_y_svs.csv', low_memory=False))
eyesight_filter = svs[(((svs['snellen_va_y'].isin([0,1]))) | (svs['vis_flg'].isin([2]))) & (svs['eventname'] == 'baseline_year_1_arm_1')]#
eyesight_filter['src_subject_id'].to_csv('/media/hcs-sci-psy-narun/ABCC_F/bad_eye_sight.csv')
len(eyesight_filter)

31

In [ ]:
# demographic
demo = pd.read_csv(tables_path+'participants.tsv', sep = '\t' ,index_col=0)
demo = demo[~demo['site'].isin(['site22','888'])]
demo.index = demo.index.str.replace('sub-', '')
demo = demo[~demo.index.duplicated(keep='first')]
# fix site for specific subjects (ids hidden):
demo.loc['***', 'site'] = 'site17'
demo.loc['***', 'site'] = 'site09'

# remove bad eye sight : 
demo_filtered = demo.loc[~demo.index.isin(eyesight_filter.index)]
demo_filtered.to_csv(tables_path + 'demo_nesi.csv')


In [ ]:
# Create 21 folds bsed on 21 sites for the one site out cross validation
group_kfold = GroupKFold(n_splits=21)
folds_list = list(group_kfold.split(demo_filtered, groups=demo_filtered['site']))
joblib.dump(folds_list, tables_path + 'Folds_inds.joblib', compress=1) 

['/media/hcs-sci-psy-narun/ABCC/fmriresults01/derivatives/ML_Tables/Folds_inds.joblib']

In [ ]:
folds_list = joblib.load('/media/hcs-sci-psy-narun/ABCC/fmriresults01/derivatives/ML_Tables/Folds_inds.joblib')
demo_filtered =  pd.read_csv(tables_path + 'demo_nesi.csv')
# Loop through each fold and print the test sites
for fold_idx, (train_idx, test_idx) in enumerate(folds_list):
    test_sites = demo_filtered.iloc[test_idx]['site'].unique()
    print(f"Fold {fold_idx + 1}: Test Sites: {test_sites}")

Fold 1: Test Sites: ['site16']
Fold 2: Test Sites: ['site04']
Fold 3: Test Sites: ['site10']
Fold 4: Test Sites: ['site13']
Fold 5: Test Sites: ['site20']
Fold 6: Test Sites: ['site03']
Fold 7: Test Sites: ['site21']
Fold 8: Test Sites: ['site14']
Fold 9: Test Sites: ['site12']
Fold 10: Test Sites: ['site06']
Fold 11: Test Sites: ['site17']
Fold 12: Test Sites: ['site02']
Fold 13: Test Sites: ['site19']
Fold 14: Test Sites: ['site15']
Fold 15: Test Sites: ['site11']
Fold 16: Test Sites: ['site09']
Fold 17: Test Sites: ['site01']
Fold 18: Test Sites: ['site18']
Fold 19: Test Sites: ['site05']
Fold 20: Test Sites: ['site08']
Fold 21: Test Sites: ['site07']
